# 기상 예측 데이터 전처리

In [3]:
import requests
import pandas as pd

API_KEY = 'Input API_KEY'
df_merge = pd.DataFrame()
    
for i in range(1,24):
    id = i
    hour = 14
    date = '2021-08-XX'
    
    # forecast 데이터 API로 불러오기
    forecasts_1 = requests.get(f'https://research-api.dershare.xyz/open-proc/cmpt/weathers/1/{id}/forecasts/{date}/{hour}', headers={
                            'Authorization': f'Bearer {API_KEY}'}).json()
    
    f1 = pd.DataFrame(forecasts_1)
    f1 = f1.drop(['fcst_time',  'precip_prob'], axis=1)
    f1['time'] = pd.to_datetime(f1['time']).dt.tz_convert('Asia/Seoul')
    f1 = f1.set_index(['time'], drop=True)
    
    # 시간을 늘리기 위한 데이터 UpSampling
    df = f1.resample(rule='H').last()
    df = df.interpolate()
    
    # 필요한 부분만 split
    df = df.reset_index()
    df = df.iloc[10:58] # date변수+1일~ date변수+2일
    
    # id, amount 변수 추가
    idx = []
    for i in range(len(df)):
        idx.append(id)
    df.insert(0, 'id', idx)
    df.insert(2, 'amount', 0.0)
    
    # 소수점 정리
    for col in df.columns[2:]:
        df[col] = df[col].round(1)
    
    # 데이터 프레임 합치기
    if(i==1):
        df_merge = df
        continue
    df_merge = pd.concat([df_merge, df])

df_merge.columns = [['id', 'time', 'amount', 'temperature', 'humidity', 'wind_dir', 'wind_spd', 'uv_idx', 'visibility', 'cloudiness', 'precip_1h']]
df_merge

,id,time,amount,temperature,humidity,dew_point,wind_dir,wind_spd,uv_idx,visibility,cloudiness,ceiling,precip_1h
10,1,2021-08-10 01:00:00+09:00,0.0,23.3,70.0,17.8,61.0,3.7,0.0,9.7,91.0,7498.1,0.0
11,1,2021-08-10 02:00:00+09:00,0.0,23.3,70.0,17.2,65.0,3.7,0.0,9.7,89.0,7498.1,0.0
12,1,2021-08-10 03:00:00+09:00,0.0,22.8,72.0,17.2,72.0,3.7,0.0,9.7,87.0,1981.2,0.0
13,1,2021-08-10 04:00:00+09:00,0.0,22.2,74.0,17.8,81.0,3.7,0.0,9.7,89.0,1981.2,0.0
14,1,2021-08-10 05:00:00+09:00,0.0,22.2,77.0,18.3,91.0,3.7,0.0,9.7,91.0,487.7,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
53,23,2021-08-11 20:00:00+09:00,0.0,25.6,71.0,20.0,307.0,5.6,0.0,16.1,33.0,9144.0,0.0
54,23,2021-08-11 21:00:00+09:00,0.0,24.4,78.0,20.0,279.0,3.7,0.0,11.3,33.0,9144.0,0.0
55,23,2021-08-11 22:00:00+09:00,0.0,23.3,84.0,20.6,311.0,3.7,0.0,11.3,31.0,9144.0,0.0
56,23,2021-08-11 23:00:00+09:00,0.0,22.2,88.0,20.0,39.0,3.7,0.0,9.7,29.0,9144.0,0.0


In [4]:
path = './data/'
df_merge.to_csv(path + "forecast.csv", index = False)